In [1]:
!pip install reverse_geocoder

StatementMeta(, 5b38096a-5856-4a1c-9741-8f63608bb3fe, 5, Finished, Available, Finished)

In [2]:
# Import necessary libraries for data processing and API calls
from pyspark.sql.functions import when, col, udf
# Import necessary libraries for data processing and API calls
from pyspark.sql.types import StringType
# Ensure the below library is installed on your fabric environment
# Import necessary libraries for data processing and API calls
import reverse_geocoder as rg
# Import necessary libraries for data processing and API calls
from datetime import date, timedelta

StatementMeta(, 5b38096a-5856-4a1c-9741-8f63608bb3fe, 6, Finished, Available, Finished)

In [3]:
# Remove this before running Data Factory Pipeline
start_date = date. today() - timedelta(7)

StatementMeta(, 5b38096a-5856-4a1c-9741-8f63608bb3fe, 7, Finished, Available, Finished)

In [4]:
# Load data into a Spark DataFrame for processing
df = spark.read.table("earthquake_events_stage2").filter(col('time') > start_date)
     

def get_country_code(lat, lon):
    """
    Retrieve the country code for a given latitude and longitude.

    Parameters:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Returns:
    str: Country code of the location, retrieved using the reverse geocoding API.

    Example:
    >>> get_country_details(48.8588443, 2.2943506)
    'FR'
    """
    coordinates = (float(lat), float(lon))
    return rg.search(coordinates)[0].get('cc')

StatementMeta(, 5b38096a-5856-4a1c-9741-8f63608bb3fe, 8, Finished, Available, Finished)

In [5]:
# Define a user-defined function (UDF) for data transformation
# registering the udfs so they can be used on spark dataframes
# Define a user-defined function (UDF) for data transformation
get_country_code_udf = udf(get_country_code, StringType())

StatementMeta(, 5b38096a-5856-4a1c-9741-8f63608bb3fe, 9, Finished, Available, Finished)

In [6]:
# adding country_code and city attributes
df_with_location = \
                df.\
# Define a user-defined function (UDF) for data transformation
                    withColumn("country_code", get_country_code_udf(col("latitude"), col("longitude")))

StatementMeta(, 5b38096a-5856-4a1c-9741-8f63608bb3fe, 10, Finished, Available, Finished)

In [7]:
# adding significance classification
df_with_location_sig_class = \
                            df_with_location.\
                                withColumn('sig_class', 
# Apply conditional transformations to clean and filter the dataset
                                            when(col("sig") < 100, "Low").\
# Apply conditional transformations to clean and filter the dataset
                                            when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
                                            otherwise("High")
                                            )
     



StatementMeta(, 5b38096a-5856-4a1c-9741-8f63608bb3fe, 11, Finished, Available, Finished)

In [8]:
# appending the data to the stage3 table
# Save the transformed data into a Delta table for querying
df_with_location_sig_class.write.mode('append').saveAsTable('earthquake_events_stage3')

StatementMeta(, 5b38096a-5856-4a1c-9741-8f63608bb3fe, 12, Finished, Available, Finished)